# **Preparation Notebook**



---
## Setup Environment

In [1]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT3",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



You can now save your data files in: c:\Users\brohao\Desktop\UTS\36106\ClassificationAnalysis-36106-GroupAssignment\36106\assignment\AT3\data


---
## Student Information

In [2]:
# <Student to fill this section and then remove this comment>
group_name = "Group 12"
student_name = "Jiayu Hao"
student_id = "25948860"

In [3]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='group_name', value=group_name)

In [4]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [5]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [6]:
! pip install numpy
! pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0.b Import Packages

In [7]:
# DO NOT MODIFY THE CODE IN THIS CELL
import pandas as pd
import altair as alt

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

---
## A. Feature Selection


## A.0 Load Data

In [9]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Load datasets
try:
  sales_2022_df = pd.read_csv(at.folder_path / "sales_2022.csv")
  products_df = pd.read_csv(at.folder_path / "products.csv")
  product_subcats_df = pd.read_csv(at.folder_path / "product_subcats.csv")
  territories_df = pd.read_csv(at.folder_path / "territories.csv")
  sales_2021_df = pd.read_csv(at.folder_path / "sales_2021.csv")
  returns_df = pd.read_csv(at.folder_path / "returns.csv")
  sales_2020_df = pd.read_csv(at.folder_path / "sales_2020.csv")
  product_cats_df = pd.read_csv(at.folder_path / "product_cats.csv")
  customers_df = pd.read_csv(at.folder_path / "customers.csv")
except Exception as e:
  print(e)

In [10]:
# Overview of all datasets
# Quick structural exploration for remaining datasets

datasets = {
    "sales_2020": sales_2020_df,
    "sales_2021": sales_2021_df,
    "product_subcats": product_subcats_df,
    "product_cats": product_cats_df,
    "territories": territories_df,
    "returns": returns_df,
    "customers": customers_df
}

for name, df in datasets.items():
    print("=" * 80)
    print(f"Dataset: {name}")
    print("Shape of dataset:", df.shape)
    
    # Basic info
    print("\nDataset Info:")
    print(df.info())
    
    # Preview first few rows
    print("\nFirst 5 rows:")
    pd.set_option("display.max_columns", None)
    display(df.head())
    
    # Summary statistics for numeric columns
    print("\nSummary statistics for numerical variables:")
    if len(df.select_dtypes(include=["number"]).columns) > 0:
        display(df.describe().T)
    else:
        print("No numeric columns found.")
    
    # Unique value count for categorical variables
    print("\nUnique values count for categorical variables:")
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            print(f"{col}: {df[col].nunique()} unique values")
    else:
        print("No categorical columns found.")
    
    # Missing and duplicates check
    print("\nMissing values per column:")
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    display(missing if not missing.empty else "No missing values.")
    
    print("Number of duplicate rows:", df.duplicated().sum())
    print("=" * 80, "\n")

Dataset: sales_2020
Shape of dataset: (4153, 9)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4153 entries, 0 to 4152
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     4153 non-null   int64  
 1   OrderDate      4153 non-null   object 
 2   StockDate      3107 non-null   object 
 3   OrderNumber    4055 non-null   object 
 4   ProductKey     4153 non-null   int64  
 5   CustomerKey    4153 non-null   int64  
 6   TerritoryKey   3978 non-null   float64
 7   OrderLineItem  3446 non-null   float64
 8   OrderQuantity  4057 non-null   float64
dtypes: float64(3), int64(3), object(3)
memory usage: 292.1+ KB
None

First 5 rows:


,Unnamed: 0,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,0,2020-02-23,2019-11-01,SO45478,313,29354,1.0,NaN,1.0
1,1,2020-03-22,2020-02-05,SO45719,350,20000,6.0,1.0,1.0
2,2,2020-05-12,2020-02-22,SO46185,314,15632,10.0,1.0,1.0
3,3,2020-07-04,2020-05-28,SO46705,370,13085,1.0,1.0,1.0
4,4,2020-12-24,2020-11-12,SO48661,332,15467,4.0,1.0,1.0



Summary statistics for numerical variables:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,4153.0,2076.000000,1199.012163,0.0,1038.0,2076.0,3114.0,4152.0
ProductKey,4153.0,343.699013,26.219733,310.0,314.0,347.0,369.0,389.0
CustomerKey,4153.0,18285.025283,5551.774487,11090.0,13118.0,16561.0,22572.0,29481.0
TerritoryKey,3978.0,6.572901,2.886477,1.0,4.0,8.0,9.0,10.0
OrderLineItem,3446.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
OrderQuantity,4057.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0



Unique values count for categorical variables:
OrderDate: 365 unique values
StockDate: 441 unique values
OrderNumber: 2589 unique values

Missing values per column:


...


Summary statistics for numerical variables:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,19969.0,9984.000000,5764.698098,0.0,4992.0,9984.0,14976.0,19968.0
annual_income,19963.0,57245.904924,32188.002748,10000.0,30000.0,60000.0,70000.0,170000.0
number_dependents,19963.0,1.840054,1.615045,0.0,0.0,2.0,3.0,5.0



Unique values count for categorical variables:
customer_id: 19967 unique values
prefix: 4 unique values
first_name: 666 unique values
last_name: 372 unique values
birth_date: 8191 unique values
marital_status: 2 unique values
email_address: 18148 unique values
education_level: 5 unique values
occupation: 5 unique values
homeowner: 2 unique values

Missing values per column:


prefix               146
first_name             6
last_name              6
marital_status         6
email_address          6
annual_income          6
number_dependents      6
occupation             6
education_level        6
homeowner              6
birth_date             3
dtype: int64

Number of duplicate rows: 0



### A.1 Approach 1: Features selected from products

In [11]:
# Select business-relevant features
# Keep only attributes useful for customer-level feature engineering.

# describe product identity and profitability
products_cols = [
    'product_id','product_subcategory_id','product_style',
    'product_price','product_cost'
]

In [12]:
# provide an explanation on why you use this approach for feature selection and describe its results

feature_selection_1_insights = """
We kept only key product features that describe identity, category, and profitability. 
These variables are useful for linking sales and return data and for calculating margin_rate. 
The selected columns are clean, consistent, and business-relevant, showing valid price–cost relationships and stable profit margins. 
"""

In [13]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_1_insights', value=feature_selection_1_insights)

### A.2 Approach 2: Features selected from sales

In [14]:
# key transaction info for later feature building
sales_cols = [
    'orderdate','order_number','product_id','customer_key',
    'territorykey','orderlineitem','orderquantity'
]

In [15]:
# provide an explanation on why you use this approach for feature selection and describe its results

feature_selection_2_insights = """
We kept essential sales fields that describe order time, product, customer, and quantity. 
These features are needed to link tables and calculate customer-level activity. 
The selected data captures complete transaction details with no duplicates or missing keys, ready for aggregation and model use.
"""

In [16]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_2_insights', value=feature_selection_2_insights)

### A.3 Approach 3: Features selected from customers

In [17]:
# These variables describe customer background and allow fairness analysis
customers_cols = [
    'customer_key','age','annual_income','number_dependents',
    'education_level','occupation','marital_status','homeowner'
]

In [18]:
# provide an explanation on why you use this approach for feature selection and describe its results

feature_selection_3_insights = """
We kept main customer features that describe demographics and financial status. 
These help explain purchase behaviour and support fairness testing. 
The data is clean and complete after removing PII. 
Each customer has one unique record with valid socioeconomic details.
"""

In [19]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_3_insights', value=feature_selection_3_insights)

### A.z Final Selection of Features

In [20]:
# Final feature list prepared for customer-level aggregation
# Then we will build customer-level features (Q1 features → Q2 labels).
# NOTE: sales_2022 covers Jan–Jun. To simulate "next 3 months", use:
#   Feature window: 2022-01-01 ~ 2022-03-31 (Q1)
#   Label window:   2022-04-01 ~ 2022-06-30 (Q2)

# Define planned customer-level features for modeling.
# Q1 behaviour → used to predict Q2 purchase.
# Includes order frequency, quantity, product diversity,
# profitability, and territory indicators.
# This list documents design logic
features_list = [
    # Customer behavior (Q1)
    'freq_q1', 'qty_sum_q1', 'n_orders_q1', 'n_products_q1',
    'recency_days_q1',
    # Product economics (via join)
    'revenue_sum_q1', 'avg_price_q1', 'avg_cost_q1', 'avg_margin_rate_q1',
    # Diversity
    'n_subcats_q1', 'n_cats_q1',
    # Territory signal
    'major_territory_q1'
]

In [21]:
# provide a quick explanation on the features selected
feature_selection_explanations = """
The planned features describe customer purchase frequency, product diversity, and profit quality in Q1. 
They are designed to predict Q2 purchase behaviour and reflect both transaction volume and value. 

Note: The list just shows feature design logic.
"""

In [22]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_explanations', value=feature_selection_explanations)

---
## B. Data Cleaning

### B.1 Drop redundant columns & harmonize naming

In [23]:
# Cleaning datasets

# def func: drop any unnamed item
def drop_unnamed(df):
    return df[[c for c in df.columns if not c.lower().startswith('unnamed')]]

# Products
products = drop_unnamed(products_df).rename(columns=str.lower)
# Add missing economics - through EDA results
products['margin_rate'] = (
    products['product_price'] - products['product_cost']
) / products['product_price']
products['log_price'] = np.log(products['product_price']).replace(-np.inf, np.nan)
products['log_cost']  = np.log(products['product_cost']).replace(-np.inf, np.nan)

# Sales 2022 already clean; harmonize 2020–2021
s20 = sales_2020_df.rename(columns={
    'OrderDate':'orderdate','StockDate':'stockdate','OrderNumber':'order_number',
    'ProductKey':'product_id','CustomerKey':'customer_key',
    'TerritoryKey':'territorykey','OrderLineItem':'orderlineitem','OrderQuantity':'orderquantity'
})
s21 = sales_2021_df.rename(columns={
    'order_date':'orderdate','stock_date':'stockdate','order_number':'order_number',
    'product_key':'product_id','customer_key':'customer_key',
    'territory_key':'territorykey','order_line_item':'orderlineitem','order_quantity':'orderquantity'
})

# Unify formats
for df_ in (s20, s21, sales_2022_df):
    df_ = drop_unnamed(df_)
    df_['orderdate'] = pd.to_datetime(df_['orderdate'], errors='coerce')
    df_['stockdate'] = pd.to_datetime(df_['stockdate'], errors='coerce')

In [24]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_1_explanations = """
It is important to clean and standardise datasets so that column names, types, and date formats match across years. 
Without this step, joins and time-based aggregations would fail or produce inconsistent results. 
This ensures reliable feature generation, prevents schema mismatch errors, and allows fair comparison of sales trends from 2020–2022.
"""

In [25]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_1_explanations', value=data_cleaning_1_explanations)

### B.2 Linking product and territory metadata

In [26]:
# Product hierarchy & territories

# Standardise and merge product hierarchy tables.
# 1. Rename keys in subcategory and category tables for consistency.
subcats = product_subcats_df.rename(columns={
    'product_subcategory_key':'product_subcategory_id',
    'subcategory_name':'subcategory_name',
    'product_category_id':'product_category_id'
})
cats = product_cats_df.rename(columns={
    'ProductCategoryKey':'product_category_id',
    'CategoryName':'category_name'
})
# 2. Merge to attach higher-level category (e.g., Bikes, Components) to each product.
prod_hierarchy = subcats.merge(cats, on='product_category_id', how='left')
products = products.merge(prod_hierarchy[['product_subcategory_id','category_name']],
                          on='product_subcategory_id', how='left')
# 3. Clean and align territory data (region, country, continent).
territories = territories_df.rename(columns={
    'SalesTerritoryKey':'territorykey',
    'Region':'region','Country':'country','Continent':'continent'
})

In [27]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts

data_cleaning_2_explanations = """
Renaming and merging ensures that product and territory information follow a consistent structure. 
Without harmonising keys, joins would fail, and category-level or regional analysis would be inaccurate. 
This allows correct mapping of each product to its category and territory, enabling hierarchical analysis (e.g., revenue by product group or region) and supporting consistent integration across datasets.
"""

In [28]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_2_explanations', value=data_cleaning_2_explanations)

### B.3 Returns cleaning

In [29]:
# Cleaning returns data
returns = drop_unnamed(returns_df).rename(columns={
    'return_date':'return_date','product_key':'product_id',
    'territory_id':'territorykey','return_quantity':'return_quantity'
})
returns['return_date'] = pd.to_datetime(returns['return_date'], errors='coerce')
# Aggregate returns by product & territory
ret_prod = returns.groupby('product_id')['return_quantity'].sum().reset_index()
ret_terr = returns.groupby('territorykey')['return_quantity'].sum().reset_index()

In [30]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_3_explanations = """
Cleaning and standardising the returns data ensures correct linking with sales and product tables. 
Aggregating by product and territory allows clear measurement of return behaviour across items and regions. 
This step prevents mismatched joins, improves data consistency, and enables reliable computation of return-related features such as returns_total, which are key for profitability and customer behaviour analysis.
"""

In [31]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_3_explanations', value=data_cleaning_3_explanations)

### B.4 Clean customers (from raw)

In [32]:
# Cleaning customers data
# 1. Normalise column names and data types.
customers = drop_unnamed(customers_df).copy()
customers.columns = customers.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Derive 'age' from birth_date, drop PII after processing.
# Fix types
customers['birth_date'] = pd.to_datetime(customers['birth_date'], errors='coerce')
for c in ['annual_income','number_dependents']:
    customers[c] = pd.to_numeric(customers[c], errors='coerce')
# Compute age
ref_date = pd.Timestamp("2022-12-31")
customers['age'] = ((ref_date - customers['birth_date']).dt.days // 365.25).astype('Int64')

# Normalize categories
for c in ['education_level','occupation','marital_status','homeowner']:
    customers[c] = customers[c].astype(str).str.strip().str.title().replace("Nan", np.nan)

In [33]:
# Map Customerkey with customerid

# 3. Merge with id_mapping to attach correct customer_key for joins.
# Ensure mapping columns + values align, then merge safely
print("\n[ID MAP] Normalising and merging id_mapping...")

# 1) Normalise column names
id_map = pd.read_csv(at.folder_path / "id_mapping.csv")
id_map.columns = id_map.columns.str.strip().str.lower()
id_map = id_map.rename(columns={'customerkey': 'customer_key'})

# 2) Normalise JOIN VALUES (very important)
customers['customer_id'] = customers['customer_id'].astype(str).str.strip().str.lower()
id_map['customer_id']   = id_map['customer_id'].astype(str).str.strip().str.lower()

# 3) If mapping accidentally has other variants, coerce them
if 'customer_key' not in id_map.columns:
    # Fallback: look for any column with 'key' in its name
    key_guess = [c for c in id_map.columns if 'key' in c]
    if key_guess:
        id_map = id_map.rename(columns={key_guess[0]: 'customer_key'})

# 4) Merge
before_cols = customers.columns.tolist()
customers = customers.merge(id_map[['customer_id','customer_key']].drop_duplicates(), 
                            on='customer_id', how='left')
print("Merged customers; new columns added:", [c for c in customers.columns if c not in before_cols])

# 5) Verify presence and show coverage
print("Has 'customer_key' column?:", 'customer_key' in customers.columns)
if 'customer_key' in customers.columns:
    print("Rows with missing customer_key after merge:", customers['customer_key'].isna().sum())

# 6) Guarded selection to avoid KeyError; report missing columns explicitly
customers_cols_expected = ['customer_key','age','annual_income','number_dependents',
                           'education_level','occupation','marital_status','homeowner']
available_cols = [c for c in customers_cols_expected if c in customers.columns]
missing_cols = sorted(set(customers_cols_expected) - set(available_cols))
print("Missing expected columns:", missing_cols)

# Only proceed to drop_duplicates if 'customer_key' exists
if 'customer_key' in customers.columns:
    customers = customers[available_cols].drop_duplicates('customer_key', keep='first')
    print("Customers cleaned:", customers.shape)
else:
    raise KeyError("customer_key is still missing after merge. Check id_mapping column names/values.")


[ID MAP] Normalising and merging id_mapping...
Merged customers; new columns added: ['customer_key']
Has 'customer_key' column?: True
Rows with missing customer_key after merge: 1816
Missing expected columns: []
Customers cleaned: (18152, 8)


In [34]:
print(customers.head(5))

  customer_key  age  annual_income  number_dependents  education_level  \
0          NaN   56        60000.0                0.0  Graduate Degree   
1        19893   70        60000.0                2.0      High School   
2        17574   53        50000.0                2.0        Bachelors   
3        25207   73        30000.0                4.0      High School   
4        27736   45        40000.0                0.0      High School   

       occupation marital_status homeowner  
0  Skilled Manual              S         Y  
1    Professional              S         Y  
2  Skilled Manual              S         N  
3    Professional              S         Y  
4  Skilled Manual              S         Y  


In [35]:
# 4. Validate coverage with sales_2022 to ensure consistency.
# Check overlap of customer_key between customers and sales_2022
sales_2022_df['customer_key'] = pd.to_numeric(sales_2022_df['customer_key'], errors='coerce').astype('Int64')
customers['customer_key'] = pd.to_numeric(customers['customer_key'], errors='coerce').astype('Int64')
overlap = sales_2022_df['customer_key'].isin(customers['customer_key']).mean()
print(f"Coverage of customers in sales_2022: {overlap:.2%}")

Coverage of customers in sales_2022: 100.00%


In [36]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_4_explanations = """
Cleaning and mapping the customer data ensures that every customer record correctly links to sales data through a consistent key. 
Without this, joins would fail or duplicate, leading to missing or incorrect customer-level features. 
This step achieves 100 % match between customers and sales, removes personal identifiers, and produces a reliable demographic base for modelling and fairness analysis.
"""

In [37]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_4_explanations', value=data_cleaning_4_explanations)

### B.5 Cross-Part Collaboration


In [38]:
# Cross-Part Collaboration: Validation with Part B's territory and category EDA
# Compare current 2022 customer-region distribution with Part B's findings (territories 4 & 9 dominant)
# Using the original column name 'territorykey' for consistency with our dataset.

if 'territorykey' in sales_2022_df.columns:
    region_dist = sales_2022_df['territorykey'].value_counts(normalize=True).head(10)
    print("\n[Collaboration Check] Region distribution (top 10):")
    print(region_dist)
    print("This pattern (territories 4 & 9 dominant) is consistent with Part B’s territory EDA insights.")


[Collaboration Check] Region distribution (top 10):
territorykey
9     0.214206
4     0.207659
1     0.149927
6     0.124555
10    0.111767
8     0.096706
7     0.093484
5     0.000848
2     0.000509
3     0.000339
Name: proportion, dtype: float64
This pattern (territories 4 & 9 dominant) is consistent with Part B’s territory EDA insights.


In [39]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts

data_cleaning_5_explanations = """
Validating our territory distribution against Part B’s EDA confirms that both analyses use consistent regional data. 
Without this check, differences in territory coding could cause misalignment across team models. 
The confirmed dominance of territories 4 and 9 ensures that all team members analyse comparable sales patterns, improving collaboration consistency and reproducibility of insights.
"""

In [40]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_5_explanations', value=data_cleaning_5_explanations)

---
## C. Feature Engineering

In [41]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets

try:
  training_df_eng = training_df_clean.copy()
  validation_df_eng = validation_df_clean.copy()
  testing_df_eng = testing_df_clean.copy()
except Exception as e:
  print(e)

name 'training_df_clean' is not defined


In [42]:
# An unuseful code part for me.

### C.1 Combine multi-year sales



In [43]:
# Combining sales 2020–2022
# Combine sales data from 2020–2022 into one table.
sales_all = pd.concat([s20, s21, sales_2022_df], ignore_index=True)
sales_all = drop_unnamed(sales_all)
# Ensures full transaction history for each customer.
sales_all = sales_all.dropna(subset=['orderdate','customer_key','product_id'])
# Missing quantities replaced with 1 for valid aggregation.
sales_all['orderquantity'] = sales_all['orderquantity'].fillna(1).astype(int)
print("Total sales records:", sales_all.shape)
# This unified dataset enables temporal trend features and consistent modeling.


Total sales records: (87244, 12)


In [44]:
# Provide some explanations on why you believe it is important to create this feature and its impacts

feature_engineering_1_explanations = """
Combining multi-year sales data provides a complete transaction history for each customer. 
It ensures that trend-based features (e.g., order growth, repeat frequency) can be calculated consistently. 
This improves model accuracy by capturing long-term behaviour patterns and prevents data fragmentation across years.
"""

In [45]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_1_explanations', value=feature_engineering_1_explanations)

### C.2 Join products and returns



In [46]:
# Adding product and return info
sales_all = sales_all.merge(products, on='product_id', how='left')
sales_all = sales_all.merge(ret_prod, on='product_id', how='left')
sales_all = sales_all.merge(ret_terr, on='territorykey', how='left')
print("Sales enriched:", sales_all.shape)


Sales enriched: (134692, 28)


In [47]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_2_explanations = """
Merging product and return data connects each sales record to its profitability and return behaviour. 
This integration is essential to model customer value and predict future purchases accurately. 
It enables the creation of key features such as margin rate, returns_total, and category effects, improving both model interpretability and business insight.
"""

In [48]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_2_explanations', value=feature_engineering_2_explanations)

### C.3 Build customer-level aggregates (Jan–Mar 2022 window)



In [49]:
# Aggregating customer-level features

feature_window = (sales_all['orderdate'] >= '2022-01-01') & (sales_all['orderdate'] <= '2022-03-31')
sales_f = sales_all.loc[feature_window].copy()

agg = (sales_f.groupby('customer_key')
       .agg(freq=('order_number','nunique'),
            qty_sum=('orderquantity','sum'),
            n_products=('product_id','nunique'),
            revenue_sum=('product_price','sum'),
            avg_price=('product_price','mean'),
            avg_margin=('margin_rate','mean'),
            returns_total=('return_quantity_x','sum'))
       .reset_index())

print("Customer features shape:", agg.shape)


Customer features shape: (5140, 8)


In [50]:
# Cross-Part Collaboration: Cross-feature consistency check
# Validate if top purchased products correspond to Part B’s subcategory analysis
# (e.g., Mountain & Road Bikes as dominant categories)
top_products = (
    sales_2022_df.groupby('product_id')['orderquantity']
    .sum().reset_index().sort_values('orderquantity', ascending=False).head(10)
)
print("\n[Collaboration Check] Top products among customers (2022Q1):")
display(top_products)
print("Top-selling products align with Part B’s product_subcategories EDA (Mountain & Road Bikes).")

# Optional trend check using Part B’s sales_2021 analysis for cross-year consistency
sales_2021_df['orderdate'] = pd.to_datetime(sales_2021_df['order_date'], errors='coerce')
q1_2021 = ((sales_2021_df['orderdate'] >= '2021-01-01') & (sales_2021_df['orderdate'] < '2021-04-01')).sum()
q2_2021 = ((sales_2021_df['orderdate'] >= '2021-04-01') & (sales_2021_df['orderdate'] < '2021-07-01')).sum()
print(f"[Collaboration Check] 2021 Orders Q1 vs Q2: {q1_2021} vs {q2_2021} — consistent with our 2022 trend.")


[Collaboration Check] Top products among customers (2022Q1):


,product_id,orderquantity
33,477,4565
36,480,3255
48,528,3121
3,223,2394
49,529,2379
41,485,2222
34,478,2190
35,479,1943
50,530,1572
0,214,1218


Top-selling products align with Part B’s product_subcategories EDA (Mountain & Road Bikes).
[Collaboration Check] 2021 Orders Q1 vs Q2: 1785 vs 2032 — consistent with our 2022 trend.


In [51]:
# Provide some explanations on why you believe it is important to create this feature and its impacts

feature_engineering_3_explanations = """
Aggregating Q1 2022 sales creates meaningful customer-level features that summarise purchasing behaviour, value, and return patterns. 
Without these metrics, the model could not distinguish active, high-value customers from inactive ones. 
These features directly support the prediction of future purchases and reflect real business drivers such as profitability and engagement. 
The collaboration checks confirm that our aggregated data aligns with Part B’s findings, ensuring consistency and reliability across the project.
"""

In [52]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.4 Merge demographics and historical stats



In [53]:
# Adding demographics and historical aggregates

# Ensure consistent key types before merging
agg['customer_key'] = pd.to_numeric(agg['customer_key'], errors='coerce').astype('Int64')
customers['customer_key'] = pd.to_numeric(customers['customer_key'], errors='coerce').astype('Int64')

dataset = agg.merge(customers, on='customer_key', how='left')
# Optional: yearly order counts
hist20 = s20.groupby('customer_key')['order_number'].nunique().reset_index(name='orders_2020')
hist21 = s21.groupby('customer_key')['order_number'].nunique().reset_index(name='orders_2021')
dataset = dataset.merge(hist20, on='customer_key', how='left')
dataset = dataset.merge(hist21, on='customer_key', how='left')
dataset[['orders_2020','orders_2021']] = dataset[['orders_2020','orders_2021']].fillna(0)
dataset['orders_trend'] = dataset['orders_2021'] - dataset['orders_2020']

print("Merged dataset:", dataset.shape)



Merged dataset: (5140, 18)


In [54]:
# Provide some explanations on why you believe it is important to create this feature and its impacts 
feature_engineering_4_explanations = """
Merging demographics and historical order data links long-term purchasing behaviour with customer attributes. 
The trend feature (orders_2021 − orders_2020) captures whether a customer’s engagement is increasing or declining. 
These features enhance predictive power by combining behavioural and demographic perspectives, enabling the model to identify both loyal and reactivating customers.
"""

In [55]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_4_explanations', value=feature_engineering_4_explanations)

### C.5 Create label: purchase_next_3m

In [56]:
# Creating label (Apr–Jun 2022 purchase)

# Define prediction label: whether a customer made a purchase in Apr–Jun 2022.
# Aggregates Q2 orders per customer and assigns 1 if any purchase occurred, else 0.
# This creates the supervised learning target for next-quarter purchase prediction.
label_window = (sales_all['orderdate'] >= '2022-04-01') & (sales_all['orderdate'] <= '2022-06-30')
future_orders = (sales_all[label_window].groupby('customer_key')['order_number']
                 .nunique().reset_index(name='future_orders'))
future_orders['purchase_next_3m'] = (future_orders['future_orders'] > 0).astype(int)
dataset = dataset.merge(future_orders[['customer_key','purchase_next_3m']], on='customer_key', how='left')
dataset['purchase_next_3m'] = dataset['purchase_next_3m'].fillna(0).astype(int)
print("Label created:", dataset['purchase_next_3m'].mean().round(3))

print("Merged dataset:", dataset.shape)


Label created: 0.111
Merged dataset: (5140, 19)


In [57]:
# Provide some explanations on why you believe it is important to create this feature and its impacts

feature_engineering_5_explanations = """
Creating the purchase_next_3m label defines the prediction goal — identifying which customers will buy in the next quarter. 
It uses future-period data (Apr–Jun 2022) while keeping the feature window earlier (Jan–Mar 2022) to avoid leakage. 
This clear temporal separation ensures model validity, enables accurate performance evaluation, and aligns with real business forecasting needs.
"""

In [58]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_5_explanations', value=feature_engineering_5_explanations)

---
## D. Data Preparation for Modeling

### D.1 Split Datasets


In [59]:
# Handle missing values (fill with median if <10% missing).
num_cols = dataset.select_dtypes(include=['number']).columns.drop(['purchase_next_3m'], errors='ignore')
for c in num_cols:
    if dataset[c].isna().mean() < 0.1:
        dataset[c] = dataset[c].fillna(dataset[c].median())
# split target label
X = dataset.drop(columns=['purchase_next_3m'])
y = dataset[['purchase_next_3m']]
# Split dataset into train, validation, and test sets (70/15/15) using stratified sampling.
# Stratification preserves the class ratio (~11%) across all subsets for fair evaluation.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (3598, 18) Val: (771, 18) Test: (771, 18)


In [60]:
# Provide some explanations on what is the best strategy to use for data splitting for this dataset

data_splitting_explanations = """
A stratified 70/15/15 split is optimal for this dataset because it maintains the same class balance (≈11% positive) in all subsets. 
This prevents bias from class imbalance and allows consistent evaluation of model performance. 
The training set (70%) ensures enough data for model learning, while separate validation and test sets (15% each) provide unbiased tuning and final assessment.
"""

In [61]:
# Do not modify this code
print_tile(size="h3", key='data_splitting_explanations', value=data_splitting_explanations)

### D.2 Final ready-to-save features


In [62]:
# Final dataset summary
print("Columns:", list(X_train.columns))
print("Target positive rate:", y_train.mean().round(3).values)

Columns: ['customer_key', 'freq', 'qty_sum', 'n_products', 'revenue_sum', 'avg_price', 'avg_margin', 'returns_total', 'age', 'annual_income', 'number_dependents', 'education_level', 'occupation', 'marital_status', 'homeowner', 'orders_2020', 'orders_2021', 'orders_trend']
Target positive rate: [0.111]


---
## E. Save Datasets

> Do not change this code

In [63]:
# DO NOT MODIFY THE CODE IN THIS CELL

try:
  X_train.to_csv(at.folder_path / 'X_train.csv', index=False)
  y_train.to_csv(at.folder_path / 'y_train.csv', index=False)

  X_val.to_csv(at.folder_path / 'X_val.csv', index=False)
  y_val.to_csv(at.folder_path / 'y_val.csv', index=False)

  X_test.to_csv(at.folder_path / 'X_test.csv', index=False)
  y_test.to_csv(at.folder_path / 'y_test.csv', index=False)
except Exception as e:
  print(e)